![](http://pic1.tsingdataedu.com/ML%E6%A1%88%E4%BE%8Bbanner.jpg)

# <font color=blue>动手搭建推荐系统必须跨过的N个坑</font>
作者：寒小阳、助教-檀越

## 作业说明

本次的作业一共有6个任务

1.任务一：在数据集data上进行3折交叉验证

2.任务二：读取文本格式，分隔符用'\t'

3.任务三：运用grid_search的属性读取相关参数

4.任务四：用algo.get_neighbors()找到距离toy_story_inner_id最近的10个id

5.任务五：使用algo.predict对用户进行推荐

## 代码

目录  
1.使用示例  
  * 1.1基本使用方法如下
  * 1.2载入自己的数据集方法
  * 1.3算法调参(让推荐系统有更好的效果)  
  
2.在我们的数据集上训练模型
  * 2.1建模和存储模型
    * 2.1.1.用协同过滤构建模型并进行预测  
      * ①movielens的例子
      * ②对用户进行推荐
    * 2.1.2.用矩阵分解进行预测
    * 2.1.3.模型存储  

    

### 1.使用示例

#### 1.1基本使用方法如下

<font color=blue>任务一：在数据集data上进行3折交叉验证</font>

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# 默认载入movielens数据集
data = Dataset.load_builtin('ml-100k')
# 试一把SVD矩阵分解
algo = SVD()
# k折交叉验证(k=3)，在数据集上测试一下效果
perf =                           # YOUR CODE HERE
#输出结果
print(perf)

#### 1.2载入自己的数据集方法

<font color=blue>任务二：读取文本格式，分隔符用'\t'</font>  
Tips：使用函数Reader

In [ ]:
import os
from surprise import Reader, Dataset
# 指定文件所在路径
file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')
# 告诉文本阅读器，文本的格式是怎么样的
reader =                        # YOUR CODE HERE
# 加载数据
data = Dataset.load_from_file(file_path, reader=reader)

#### 1.3算法调参(让推荐系统有更好的效果)

这里实现的算法用到的算法无外乎也是SGD等，因此也有一些超参数会影响最后的结果，我们同样可以用sklearn中常用到的网格搜索交叉验证(GridSearchCV)来选择最优的参数。简单的例子如下所示：

<font color=blue>任务三：运用grid_search的属性读取相关参数</font>  

In [ ]:
from surprise.model_selection import GridSearchCV   
# 定义好需要优选的参数网格
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
# 使用网格搜索交叉验证
grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE', 'FCP'], cv=3)
# 在数据集上找到最好的参数
data = Dataset.load_builtin('ml-100k')
grid_search.fit(data)
# 输出调优的参数组 
# 输出最好的RMSE结果
                                            # YOUR CODE HERE

# 输出对应最好的RMSE结果的参数
                                            # YOUR CODE HERE

# 最好的FCP得分
                                           # YOUR CODE HERE

# 对应最高FCP得分的参数
                                           # YOUR CODE HERE

### 2.在我们的数据集上训练模型

#### 2.1建模和存储模型

##### 2.1.1.用协同过滤构建模型并进行预测

###### ①movielens的例子

In [ ]:
# 可以使用上面提到的各种推荐系统算法
from surprise import SVD
from surprise import Dataset
from surprise import print_perf
from surprise.model_selection import cross_validate

# 默认载入movielens数据集
data = Dataset.load_builtin('ml-100k')
# 试一把SVD矩阵分解
algo = SVD()
# k折交叉验证(k=3)，在数据集上测试一下效果
perf = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
#输出结果
print_perf(perf)

以下的程序段告诉大家如何在协同过滤算法建模以后，根据一个item取回相似度最高的item，主要是用到algo.get_neighbors()这个函数

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import os
import io

from surprise import KNNBaseline
from surprise import Dataset

In [ ]:
def read_item_names():
    """
    获取电影名到电影id 和 电影id到电影名的映射
    """

    file_name = (os.path.expanduser('~') +
                 '/.surprise_data/ml-100k/ml-100k/u.item')
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]
    return rid_to_name, name_to_rid

In [ ]:
# 首先，用算法计算相互间的相似度
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

In [ ]:
# 获取电影名到电影id 和 电影id到电影名的映射
rid_to_name, name_to_rid = read_item_names()

In [ ]:
# 拿出来Toy Story这部电影对应的item id
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

<font color=blue>任务四：用algo.get_neighbors()找到距离toy_story_inner_id最近的10个id</font>  

In [ ]:
# 找到最近的10个邻居
                                        # YOUR CODE HERE

# 从近邻的id映射回电影名称
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

print()
print('The 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)

###### ②对用户进行推荐

<font color=blue>任务五：使用algo.predict对用户进行推荐</font>  
Tips：注意使用rid_to_name和to_raw_iid

In [ ]:
user_inner_id = 1000
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for movie in items:
    print(        # YOUR CODE HERE        )

##### 2.1.2.用矩阵分解进行预测

In [ ]:
### 使用SVDpp
from surprise import SVDpp
from surprise import Dataset

#file_path = os.path.expanduser('./ratings.dat')
# 指定文件格式
#reader = Reader(line_format='user item rating timestamp', sep='::')
# 从文件读取数据
#movie_data = Dataset.load_from_file(file_path, reader=reader)
# 构建数据集和建模
algo = SVDpp()
#trainset = movie.build_full_trainset()
algo.fit(trainset)

In [ ]:
user_inner_id = 1000
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for movie in items:
    print(algo.predict(algo.trainset.to_raw_uid(user_inner_id), algo.trainset.to_raw_iid(movie), r_ui=5), algo.trainset.to_raw_iid(movie))

##### 2.1.3.模型存储

In [ ]:
import surprise
surprise.dump.dump('./recommendation.model', algo=algo)
# 可以用下面的方式载入
algo = surprise.dump.load('./recommendation.model')

作者：寒小阳、助教-檀越

本文档版权归稀牛学院所有




![](http://pic1.tsingdataedu.com/%E7%A8%80%E7%89%9B%20x%20%E7%BD%91%E6%98%93.png)